# Import

## Libraries

In [ ]:
import warnings
import os
import yaml
from pathlib import Path
from dotenv import load_dotenv
from pydantic import BaseModel, Field, PositiveInt
from typing import List, Optional
from IPython.display import display, Markdown

from crewai import Agent, Task, Crew
from crewai_tools import FileReadTool, DirectoryReadTool

load_dotenv()  # take environment variables from .env
warnings.filterwarnings("ignore")

# os.environ["OPENAI_MODEL_NAME"] = "gpt-4o"

## Conversation Transcript

Output from `pdf_to_md.ipynb`:

In [ ]:
transcript_content = Path("data/conversation_01.txt").read_text()
print(transcript_content[:100])

CSR: Good morning, thank you for calling ShiokAh Insurance. This is Sarah speaking. How may I help you today?

Customer: Hi ah, I want to enquire about travel insurance. Planning to go Japan next month.

CSR: I'll be happy to help you with that. Before we look at the options, could you share some information about yourself and your travel plans?

Customer: Can, can. What you need to know?

CSR: Let me start with your age, please?

Customer: I'm 62 this year.

CSR: Thank you. And do you have any 


## Tasks & Agents YAML files

In [3]:
# Define file paths for YAML configurations
files = {
    "agents": "src/recsys/config/agents.yaml",
    "tasks": "src/recsys/config/tasks.yaml",
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, "r") as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs["agents"]
tasks_config = configs["tasks"]

In [4]:
print(
    "agents_config contains:",
    ", ".join(agents_config.keys()),
    "\ntasks_config contains:",
    ", ".join(tasks_config.keys()),
)

agents_config contains: conversation_transcript_agent, underwriting_report_agent 
tasks_config contains: process_transcript_task, generate_underwriting_report_task


## Tools (Agent Usage)

Got `FileReadTool()` to work, thanks to this [article](https://lablab.ai/t/crewai-multi-agent-system)!

In [5]:
directory_read_tool = DirectoryReadTool(directory="./data")
file_read_tool = FileReadTool()


# read_conversation = FileReadTool(file_path="./data/conversation_01.txt")
# read_income = FileReadTool(file_path="./data/income_travel_insurance_2024.txt")
# read_ge = FileReadTool(file_path="data/ge_travel_insurance_2024.txt")

Certain policies (even in Markdown) will give this error:  `Error: 'charmap' codec can't decode byte 0x81 in position 1886: character maps to <undefined>`  
This occurs because the AI agent attempting to read the file is using a character encoding incompatible with the text file's encoding. Specifically:

0x81 is a byte value that may not be defined in the default character map (often cp1252 or similar) the AI agent is using.
The error suggests the text file might be encoded in a different character set (e.g., UTF-8, ISO-8859-1, or Shift-JIS), which contains characters outside the expected range for the chosen codec.

**My solution: Fix the source code to use utf-8.**  
To specify the encoding when reading the file in the FileReadTool class, the _run method was modified to include the encoding argument in the open() function call.   
See this [link](https://chatgpt.com/share/673eeca7-0654-8010-aa31-31a1c5996d27) for details.

# Pydantic Classes

In [6]:
class CustomerInfo(BaseModel):
    age: PositiveInt = Field(..., description="Age of the customer")
    medical_conditions: Optional[List[str]] = Field(
        default_factory=list, description="List of pre-existing medical conditions"
    )
    destination: str = Field(..., description="Travel destination")
    coverage_factors: List[str] = Field(
        default_factory=list, description="Factors affecting coverage"
    )
    policy_customization_factors: List[str] = Field(
        default_factory=list, description="Factors for policy customization"
    )
    coverage_priority: List[str] = Field(
        default_factory=list,
        description="Customer's priorities for coverage, e.g., destination coverage, PA benefits",
    )


class CustomerInfoList(BaseModel):
    customers: List[CustomerInfo] = Field(default_factory=list)

# Create Crew, Agents and Tasks

[Agent attributes](https://docs.crewai.com/concepts/agents) for reference.  
Tools can be assigned when specifying Agent:

In [7]:
conversation_transcript_agent = Agent(
    config=agents_config["conversation_transcript_agent"],
    tools=[directory_read_tool, file_read_tool],
    verbose=True,
)

underwriting_report_agent = Agent(
    config=agents_config["underwriting_report_agent"],
    tools=[directory_read_tool, file_read_tool],
    verbose=True,
)

[Task attributes](https://docs.crewai.com/concepts/tasks) for reference.

In [8]:
process_transcript_task = Task(
    config=tasks_config["process_transcript_task"],
    agent=conversation_transcript_agent,
    output_pydantic=CustomerInfoList,
)

generate_underwriting_report_task = Task(
    config=tasks_config["generate_underwriting_report_task"],
    context=[process_transcript_task],
    agent=underwriting_report_agent,
    async_execution=True,
)

[Crew attributes](https://docs.crewai.com/concepts/crews) for reference.  

The `process` parameter controls how tasks are executed by the crew. The options include:

* Sequential (Default): Tasks are performed one after the other in a specific order.
* Hierarchical: One agent acts as a manager, delegating tasks to other agents while maintaining an overarching memory of the tasks.
* Parallel: Tasks are executed concurrently, allowing multiple tasks to run at the same time.


In [9]:
recsys_crew = Crew(
    agents=[conversation_transcript_agent, underwriting_report_agent],
    tasks=[
        process_transcript_task,
        generate_underwriting_report_task,
    ],
    verbose=True,
    process="sequential",
)

In [10]:
result = recsys_crew.kickoff()
display(Markdown(result.raw))

# Agent: Conversation Transcript Analyst (Travel Insurance)
## Task: This task involves analyzing a conversation transcript between a potential customer interested in travel insurance and a customer service representative from ShiokAh Insurance. The goal is to extract key customer details from the transcript, such as age, medical conditions, destination, coverage preferences, customization requests and coverage priorities. This extracted information will be organized into a Python list and used by an underwriter to determine policy suitability.


# Agent: Conversation Transcript Analyst (Travel Insurance)
## Thought: Thought: To analyze the conversation transcript and extract the necessary details, I first need to locate the file 'conversation_01.txt' in the directory. I will use the "List files in directory" tool to list the contents of the directory to confirm the file's presence.
## Using tool: List files in directory
## Tool Input: 
"{}"
## Tool Output: 
File paths: 
-./data/conver

```markdown
# Income_Travel_Insurance

## Coverage Analysis

### Customer Information:
- **Customer Age**: 62
- **Companion Age**: 60
- **Customer Medical Conditions**: High Blood Pressure, Diabetes
- **Companion Medical Conditions**: None
- **Destination**: Japan
- **Travel Period**: 2024-03-25 to 2024-04-08
- **Coverage Priorities**:
  - **Primary**: Medical Coverage
  - **Secondary**: Baggage Protection, Trip Cancellation Coverage
- **Medical Coverage Preference**: High (up to $250,000)
- **Luggage Details**:
  - **High Value Items**: None
  - **Normal Contents**: Clothes, Electronics (Phone, Camera)
- **Policy Type Preference**: Undecided (Couple or Separate)
- **Additional Notes**: Blood Pressure consideration applicable.

---

### Policy Analysis and Customer Fit:
#### Medical Coverage:
- **Coverage Amount Available**:
  - Classic Plan: $300,000 (customer's age under 'Medical Expenses Overseas' section).
  - Preferred Plan: Up to $1,000,000 (includes COVID-19 medical expenses for adults under 70 years).
  - Enhanced PreX Travel Insurance: Covers pre-existing medical conditions but not part of the base plans.
- **Coverage Exclusions**:
  - Standard plans (Classic, Deluxe, Preferred) exclude pre-existing medical conditions (e.g., diabetes and high blood pressure).
  - Enhanced PreX is necessary for this customer to have conditions like diabetes and hypertension covered.
- **Premiums**:
  - Depends on plan selection (Classic, Deluxe, or Preferred) and customization options such as Enhanced PreX.

#### Baggage Protection:
- **Coverage Amount Available**:
  - Baggage Loss/Damage:
    - Classic Plan: $3,000
    - Deluxe Plan: $5,000
    - Preferred Plan: $8,000
    - Sub-limits apply (e.g., laptops capped at $1,000 per claimable incident).
  - Baggage Delay:
    - Classic Plan: Up to $1,000 for overseas delays.
    - Preferred Plan: Up to $2,000 for overseas delays.
  - **High-Value Electronics**:
    - Electronics (phone, camera) would fall under the 'Other Items' sub-category. Coverage capped at $500/classic.

#### Trip Cancellation Coverage:
- **Coverage Amount Available**:
  - Classic Plan: $5,000 overall cancellation limit.
  - Preferred Plan: Up to $15,000 based on trip cancellation/shortening due to stipulated circumstances like illness.
  - COVID-19-related cancellations require the event to occur within 30 days prior to departure.

---

### Underwriting Considerations:
- **Exclusions**:
  - Diabetes and High Blood Pressure do not qualify under base plans.
  - Requires Enhanced PreX for medical claims related to these pre-existing conditions.
- **Policy Customization Options**:
  - Enhanced PreX is recommended for this customer to address pre-existing medical condition coverage gaps.
  - Preferred Plan would align better with the customer's "high" medical coverage preferences (up to $1,000,000 and COVID-19 medical expenses included).
- **Premium Affordability**:
  - Higher-tier plans (Preferred) with Enhanced PreX will incur significant premium costs due to added benefits and customization.

---

## Expert Opinion on Coverage Fit
The Income Travel Insurance **Enhanced PreX Preferred Plan** can potentially be a good fit for the following reasons:
1. **Primary Priority**: High medical coverage preferences can be met with Preferred coverage tiers (up to $1,000,000 overseas medical expenses) and optional Enhanced PreX to address pre-existing medical conditions like diabetes and hypertension.
2. **Secondary Priority**: Reasonable baggage protection up to $8,000 and trip cancellation limits of up to $15,000 align well with customer priorities for travel inconvenience coverage.
3. **Destination Requirements**: Comprehensive coverage for medical incidents and associated costs in Japan, a destination known for high medical expenses.

However, the higher premiums associated with the Preferred Plan and Enhanced PreX customization may be a consideration for the couple. Additionally, clear customer communication on claim limits, exclusions (e.g., non-compliance with required documentation), and sub-categories (e.g., limits for electronics) is essential to manage expectations.

### Suitability Summary:
The Preferred Plan with Enhanced PreX customization is **well-suited** to meet the customer's coverage needs. However, the high premiums should be weighed, and both customers could consider a unified 'Couple Policy' for simplicity and potential cost-efficiency.
```

This report satisfies the requested criteria by providing a structured coverage breakdown, referencing the policy details for benefits and exclusions, and offering an expert opinion on how well the customer's needs align with the policy terms.

In [11]:
print(recsys_crew.usage_metrics)

total_tokens=26028 prompt_tokens=23130 completion_tokens=2898 successful_requests=10


In [19]:
type(result.raw)

str